# BentoML Example:  Keras Text Classification

[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment. 

This notebook demonstrates how to use BentoML to turn a Keras model into a docker image containing a REST API server serving this model, how to use your ML service built with BentoML as a CLI tool, and how to distribute it a pypi package.

This notebook is built based on Keras's IMDB LSTM tutorial [here](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py).

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=keras&ea=keras-text-classification&dt=keras-text-classification)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!pip install bentoml
!pip install tensorflow==1.14.0
!pip install numpy

In [3]:
from __future__ import absolute_import, division, print_function

import numpy as np
import tensorflow as tf
print("Tensorflow Version: %s" % tf.__version__)

from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

import bentoml
print("BentoML Version: %s" % bentoml.__version__)

Tensorflow Version: 1.14.0
BentoML Version: 0.8.3+47.g7af5a8f


In [4]:
max_features = 1000
maxlen = 80 # cut texts after this number of words (among top max_features most common words)
batch_size = 300
index_from=3 # word index offset

# Prepare Dataset
Download the IMDB dataset

In [5]:
# A dictionary mapping words to an integer index
imdb.load_data(num_words=max_features)
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+index_from) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown

# Use decode_review to look at original review text in training/testing data
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(encoded_text):
    return ' '.join([reverse_word_index.get(i, '?') for i in encoded_text])

In [6]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features, index_from=index_from)

In [7]:
x_train = sequence.pad_sequences(x_train,
                                 value=word_index["<PAD>"],
                                 padding='post',
                                 maxlen=maxlen)

x_test = sequence.pad_sequences(x_test,
                                value=word_index["<PAD>"],
                                padding='post',
                                maxlen=maxlen)

# Model Training & Evaluation

In [8]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         128000    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 259,713
Trainable params: 259,713
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3, # for demo purpose :P
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 10s 419us/sample - loss: 0.4412 - acc: 0.8008 - val_loss: 0.4132 - val_acc: 0.8128
Epoch 2/3
25000/25000 [==============================] - 10s 385us/sample - loss: 0.4187 - acc: 0.8144 - val_loss: 0.4133 - val_acc: 0.8132
Epoch 3/3
25000/25000 [==============================] - 10s 380us/sample - loss: 0.4122 - acc: 0.8169 - val_loss: 0.4100 - val_acc: 0.8136


In [12]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)

print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 2s 80us/sample - loss: 0.4100 - acc: 0.8136
Test score: 0.40997193634510043
Test accuracy: 0.81356


## Define BentoService for model serving

In [32]:
%%writefile keras_text_classification_service.py
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence, text
from bentoml import api, env, BentoService, artifacts
from bentoml.artifact import KerasModelArtifact, PickleArtifact
from bentoml.adapters import JsonInput


max_features = 1000

@artifacts([
    KerasModelArtifact('model'),
    PickleArtifact('word_index')
])
@env(pip_dependencies=['tensorflow==1.14.0', 'numpy', 'pandas'])
class KerasTextClassificationService(BentoService):
   
    def word_to_index(self, word):
        if word in self.artifacts.word_index and self.artifacts.word_index[word] <= max_features:
            return self.artifacts.word_index[word]
        else:
            return self.artifacts.word_index["<UNK>"]
    
    def preprocessing(self, text_str):
        sequence = text.text_to_word_sequence(text_str)
        return list(map(self.word_to_index, sequence))
    
    @api(input=JsonInput())
    def predict(self, parsed_jsons):
        input_datas = [self.preprocessing(parsed_json['text']) for parsed_json in parsed_jsons]
        input_datas = sequence.pad_sequences(input_datas,
                                             value=self.artifacts.word_index["<PAD>"],
                                             padding='post',
                                             maxlen=80)

        return self.artifacts.model.predict_classes(input_datas).T[0]

Overwriting keras_text_classification_service.py


## Save BentoService to file archive

In [33]:
# 1) import the custom BentoService defined above
from keras_text_classification_service import KerasTextClassificationService

# 2) `pack` it with required artifacts
bento_svc = KerasTextClassificationService()
bento_svc.pack('model', model)
bento_svc.pack('word_index', word_index)

# 3) save your BentoSerivce
saved_path = bento_svc.save()

[2020-08-04 12:52:19,102] INFO - Detect BentoML installed in development model, copying local BentoML module file to target saved bundle path
running sdist
running egg_info
writing BentoML.egg-info/PKG-INFO
writing dependency_links to BentoML.egg-info/dependency_links.txt
writing entry points to BentoML.egg-info/entry_points.txt
writing requirements to BentoML.egg-info/requires.txt
writing top-level names to BentoML.egg-info/top_level.txt
reading manifest file 'BentoML.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'


no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check
creating BentoML-0.8.3+47.g7af5a8f
creating BentoML-0.8.3+47.g7af5a8f/BentoML.egg-info
creating BentoML-0.8.3+47.g7af5a8f/bentoml
creating BentoML-0.8.3+47.g7af5a8f/bentoml/adapters
creating BentoML-0.8.3+47.g7af5a8f/bentoml/artifact
creating BentoML-0.8.3+47.g7af5a8f/bentoml/cli
creating BentoML-0.8.3+47.g7af5a8f/bentoml/clipper
creating BentoML-0.8.3+47.g7af5a8f/bentoml/configuration
creating BentoML-0.8.3+47.g7af5a8f/bentoml/configuration/__pycache__
creating BentoML-0.8.3+47.g7af5a8f/bentoml/handlers
creating BentoML-0.8.3+47.g7af5a8f/bentoml/marshal
creating BentoML-0.8.3+47.g7af5a8f/bentoml/saved_bundle
creating BentoML-0.8.3+47.g7af5a8f/bentoml/server
creating BentoML-0.8.3+47.g7af5a8f/bentoml/utils
creating BentoML-0.8.3+47.g7af5a8f/bentoml/yatai
creating BentoML-0.8.3+47.g7af5a8f/bentoml/yatai/client
creating BentoML-0.8.3+47.g7af5a8f/bentoml/yatai/deployment
creating BentoML-0.8.3+47.g7af5a8f/bentoml/yatai/dep

### Test packed BentoML service

In [34]:
bento_svc.predict([{"text": "the best movie I have ever seen"}, {"text":"This is a bad movie"}])

array([1, 0], dtype=int32)

# Load BentoML Service from archive

In [14]:
import bentoml

loaded_bento_svc = bentoml.load(saved_path)

[2019-10-14 17:11:11,437] WARNING - Module `text_classification_service` already loaded, using existing imported module.


W1014 17:11:11.451318 4541836608 deprecation_wrapper.py:119] From /Users/chaoyuyang/anaconda3/envs/bentoml-dev/lib/python3.7/site-packages/bentoml/artifact/keras_model_artifact.py:104: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.

W1014 17:11:11.467543 4541836608 deprecation.py:506] From /Users/chaoyuyang/anaconda3/envs/bentoml-dev/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1014 17:11:11.468544 4541836608 deprecation.py:506] From /Users/chaoyuyang/anaconda3/envs/bentoml-dev/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Orthogonal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed 

In [15]:
loaded_bento_svc.predict([{ "text": "the best movie I have ever seen" }])

array([[1]], dtype=int32)

## Using BentoService with BentoML CLI tool

**We can use `bentoml get` command to retrieve BentoService basic information**

In [14]:
!bentoml get KerasTextClassificationService

BENTO_SERVICE                                         AGE                         APIS                  ARTIFACTS
KerasTextClassificationService:20200211173435_F31009  5 minutes and 58.2 seconds  predict<JsonHandler>  model<KerasModelArtifact>, word_index<PickleArtifact>
KerasTextClassificationService:20200206112311_4FA99B  5 days and 6 hours          predict<JsonHandler>  model<KerasModelArtifact>, word_index<PickleArtifact>


When provide `version` info, `bentoml get` will retrieve additional metadata

In [35]:
!bentoml get KerasTextClassificationService:latest

[2020-08-04 12:52:51,811] INFO - Getting latest version KerasTextClassificationService:20200804125206_4639A5
{
  "name": "KerasTextClassificationService",
  "version": "20200804125206_4639A5",
  "uri": {
    "type": "LOCAL",
    "uri": "/home/bentoml/bentoml/repository/KerasTextClassificationService/20200804125206_4639A5"
  },
  "bentoServiceMetadata": {
    "name": "KerasTextClassificationService",
    "version": "20200804125206_4639A5",
    "createdAt": "2020-08-04T04:52:19.970157Z",
    "env": {
      "condaEnv": "name: bentoml-KerasTextClassificationService\nchannels:\n- defaults\ndependencies:\n- python=3.6.10\n- pip\n",
      "pipDependencies": "tensorflow==1.14.0\nbentoml==0.8.3\nnumpy\npandas",
      "pythonVersion": "3.6.10",
      "dockerBaseImage": "bentoml/model-server:0.8.3"
    },
    "artifacts": [
      {
        "name": "model",
        "artifactType": "KerasModelArtifact"
      },
      {
        "name": "word_index",
        "artifactType": "PickleArtifact"
      }
 

`bentoml run` provides a quick way to get prediction result

In [2]:
!bentoml run KerasTextClassificationService:latest predict --input='{"text": "bad movie"}'

[2020-08-04 12:54:48,270] INFO - Getting latest version KerasTextClassificationService:20200804125206_4639A5
[2020-08-04 12:54:49,229] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-08-04 12:54:49,260] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.3, but loading from BentoML version 0.8.3+47.g7af5a8f
2020-08-04 12:54:50.476102: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2020-08-04 12:54:50.491479: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-04 12:54:50.491867: I tensorflow/core/common_runtime/gpu/gpu_device

#### Run REST API server locally

In [3]:
!bentoml serve KerasTextClassificationService:latest --enable-microbatch

[2020-08-04 12:55:23,028] INFO - Getting latest version KerasTextClassificationService:20200804125206_4639A5
[2020-08-04 12:55:23,028] INFO - Starting BentoML API server in development mode..
[2020-08-04 12:55:24,431] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-08-04 12:55:24,450] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.3, but loading from BentoML version 0.8.3+47.g7af5a8f
2020-08-04 12:55:25.644085: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2020-08-04 12:55:25.663186: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA no

### Send prediction request to REST API server

*Run the following command in terminal to make a HTTP request to the API server*
```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '{"text": "best movie ever"}' \
localhost:5000/predict
```

# "pip install" a BentoML archive

BentoML user can directly pip install saved BentoML archive with `pip install $SAVED_PATH`,  and use it as a regular python package.

In [38]:
!pip install --quiet {saved_path}

In [5]:
import KerasTextClassificationService

installed_svc = KerasTextClassificationService.load()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [6]:
installed_svc.predict([{'text': 'the best movie I have ever seen'}])

array([1], dtype=int32)

#### Additional CLI access from PyPI package

`pip install $SAVED_PATH` also installs a CLI tool for accessing the BentoML service

In [22]:
!KerasTextClassificationService --help

Usage: KerasTextClassificationService [OPTIONS] COMMAND [ARGS]...

  BentoML CLI tool

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  info            List APIs
  open-api-spec   Display OpenAPI/Swagger JSON specs
  run             Run API function
  serve           Start local rest server
  serve-gunicorn  Start local gunicorn server


### Print model service information:

In [23]:
!KerasTextClassificationService info

[2020-02-11 17:51:21,509] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-11 17:51:21,518] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+5.g3cbab90
{
  "name": "KerasTextClassificationService",
  "version": "20200211173435_F31009",
  "created_at": "2020-02-12T01:35:00.448136Z",
  "env": {
    "conda_env": "name: bentoml-KerasTextClassificationService\nchannels:\n- defaults\

### Run 'predict' api with json data:

In [1]:
!KerasTextClassificationService run predict --input='{"text": "bad movie"}'

2020-08-04 12:57:07.960998: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2020-08-04 12:57:07.975227: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-04 12:57:07.975603: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: GeForce GTX 1060 major: 6 minor: 1 memoryClockRate(GHz): 1.6705
pciBusID: 0000:01:00.0
2020-08-04 12:57:07.975784: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.1
2020-08-04 12:57:07.977201: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10
2020-08-04 12:57:07.978633: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcufft.

## Build docker container with BentoService

BentoService bundles generated `Dockerfile`. To build docker image, you simply navigate to BentoService directory and run `docker build` from there

In [ ]:
!docker build --quiet -t keras-text-classify {saved_path}

In [48]:
!docker run -p 5000:5000 keras-text-classify

[2020-02-12 19:57:53,686] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2020-02-12 19:57:54 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2020-02-12 19:57:54 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)
[2020-02-12 19:57:54 +0000] [1] [INFO] Using worker: sync
[2020-02-12 19:57:54 +0000] [8] [INFO] Booting worker with pid: 8
[2020-02-12 19:57:54 +0000] [9] [INFO] Booting worker with pid: 9
[2020-02-12 19:57:54 +0000] [10] [INFO] Booting worker with pid: 10
[2020-02-12 19:57:54,287] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+5.g3cbab90
[2020-02-12 19:57:54,411] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+5.g3cbab90
[2020-02-12 19:57:54,451] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML

## Deploy BentoService as REST API server to the cloud
BentoML support deployment to multiply cloud provider services, such as AWS Lambda, AWS Sagemaker, Google Cloudrun and etc. You can find the full list and guide on the documentation site at https://docs.bentoml.org/en/latest/deployment/index.html
For this project, we are going to deploy to AWS Sagemaker

**`bentoml sagemaker deploy` offers a single command to deployment**

In [55]:
!bentoml sagemaker deploy keras-text-classify -b KerasTextClassificationService:latest \
    --api-name predict --verbose

[2020-02-12 12:06:56,707] DEBUG - Using BentoML with local Yatai server
[2020-02-12 12:06:56,802] DEBUG - Upgrading tables to the latest revision
Deploying Sagemaker deployment |[2020-02-12 12:06:57,890] DEBUG - Created temporary directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-kdbp6qz8
-[2020-02-12 12:06:58,120] DEBUG - Getting docker login info from AWS
[2020-02-12 12:06:58,120] DEBUG - Building docker image: 192023623294.dkr.ecr.us-west-2.amazonaws.com/kerastextclassificationservice-sagemaker:20200211173435_F31009
\[2020-02-12 12:06:58,765] INFO - Step 1/11 : FROM continuumio/miniconda3:4.7.12
[2020-02-12 12:06:58,765] INFO - 

-[2020-02-12 12:06:59,697] INFO -  ---> 406f2b43ea59

[2020-02-12 12:06:59,697] INFO - Step 2/11 : EXPOSE 8080
[2020-02-12 12:06:59,697] INFO - 

[2020-02-12 12:06:59,698] INFO -  ---> Using cache

[2020-02-12 12:06:59,698] INFO -  ---> 7c8096d6922f

[2020-02-12 12:06:59,698] INFO - Step 3/11 : RUN set -x      && apt-get upd

Use `bentoml sagemaker list` to display all sagemaker deployments with BentoML

In [ ]:
!bentoml sagemaker list

To get the latest deployment state and information, use `bentoml sagemaker get` command

In [57]:
!bentoml sagemaker get keras-text-classify

{
  "namespace": "bobo",
  "name": "keras-text-classify",
  "spec": {
    "bentoName": "KerasTextClassificationService",
    "bentoVersion": "20200211173435_F31009",
    "operator": "AWS_SAGEMAKER",
    "sagemakerOperatorConfig": {
      "region": "us-west-2",
      "instanceType": "ml.m4.xlarge",
      "instanceCount": 1,
      "apiName": "predict"
    }
  },
  "state": {
    "state": "RUNNING",
    "infoJson": {
      "EndpointName": "bobo-keras-text-classify",
      "EndpointArn": "arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-text-classify",
      "EndpointConfigName": "bobo-keras-text-c-KerasTextClassificat-20200211173435-F31009",
      "ProductionVariants": [
        {
          "VariantName": "bobo-keras-text-c-KerasTextClassificat-20200211173435-F31009",
          "DeployedImages": [
            {
              "SpecifiedImage": "192023623294.dkr.ecr.us-west-2.amazonaws.com/kerastextclassificationservice-sagemaker:20200211173435_F31009",
              "ResolvedIm

We will use AWS CLI tool to test the deployment with sample data

In [58]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name bobo-keras-text-classify \
--body '{"text": "best movie ever"}' --content-type application/json output.json && cat output.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "bobo-keras-text-c-KerasTextClassificat-20200211173435-F31009"
}
[[1]]

`bentoml sagemaker delete` will delete the sagemaker deployment and related resources

In [59]:
!bentoml sagemaker delete keras-text-classify

Successfully deleted AWS Sagemaker deployment "keras-text-classify"
